In [8]:
pip install transformers datasets torch scikit-learn

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/d7/84/0df6c5981f5fc722381662ff8cfbdf8aad64bec875f75d80b55bfef394ce/datasets-3.2.0-py3-none-any.whl.metadata
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.23.2 from https://files.pythonhosted.org/packages/ea/da/6c2bea5327b640920267d3bf2c9fc114cfbd0a5de234d81cda80cc9e33c8/huggingface_hub-0.28.1-py3-none-any.whl.metadata
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
   ---------------------------------------- 0.0/464.1 kB ? eta -:--:--
    --------------------------------------- 10.2/464.1 kB ? eta -:--:--
   -- ------------------------------------ 30.7/464.1 kB 330.3 kB/s eta 0:00:02
   --- ----------------------------------- 41.0/464.1 kB 330.3 kB/s eta 0:00:02
   ------ -------------------------------- 71.7/464.1 kB 438.9 kB/s eta 0:00:01
   ------------------- -------------------- 225.3/464.1 kB 1.1 MB/s e

In [7]:
!pip install fsspec==2023.5.0



  Obtaining dependency information for fsspec==2023.5.0 from https://files.pythonhosted.org/packages/ec/4e/397b234a369df06ec782666fcdf9791d125ca6de48729814b381af8c6c03/fsspec-2023.5.0-py3-none-any.whl.metadata
  Using cached fsspec-2023.5.0-py3-none-any.whl.metadata (6.7 kB)
Using cached fsspec-2023.5.0-py3-none-any.whl (160 kB)


In [5]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


In [6]:
# Load Dataset
df = pd.read_csv("final_data.csv")


In [7]:
# Ensure correct column names
df.columns = ["text", "label"]

In [8]:
# Convert text to string and label to integer
df["text"] = df["text"].astype(str)  # Convert all text to string
df["label"] = df["label"].astype(int)  # Convert labels to integers

In [9]:
# Train-Test Split
train_texts, val_texts, train_labels, val_labels = train_test_split(df["text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42)


In [10]:
# Load Tokenizer for XLM-RoBERTa
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [11]:
# Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

In [12]:
# ✅ Convert Data to Hugging Face Dataset Format (Fixed)
train_data = Dataset.from_dict({"text": list(map(str, train_texts)), "label": list(map(int, train_labels))})
val_data = Dataset.from_dict({"text": list(map(str, val_texts)), "label": list(map(int, val_labels))})

In [13]:
# Tokenize Dataset
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/1581 [00:00<?, ? examples/s]

Map:   0%|          | 0/396 [00:00<?, ? examples/s]

In [15]:
from transformers import XLMRobertaForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained model
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
pip install accelerate>=0.26.0


Note: you may need to restart the kernel to use updated packages.


In [16]:
# Training Arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)


C:\Users\golla\anaconda3\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# Trainer Object
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
)

C:\Users\golla\AppData\Local\Temp\ipykernel_4928\1682246292.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
# Train Model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.280634
2,No log,0.307996
3,0.303400,0.255545


TrainOutput(global_step=594, training_loss=0.2914544757367786, metrics={'train_runtime': 51536.6669, 'train_samples_per_second': 0.092, 'train_steps_per_second': 0.012, 'total_flos': 1247935735572480.0, 'train_loss': 0.2914544757367786, 'epoch': 3.0})

In [19]:
# Save Model
model.save_pretrained("./fake-news-multilingual")
tokenizer.save_pretrained("./fake-news-multilingual")

('./fake-news-multilingual\\tokenizer_config.json',
 './fake-news-multilingual\\special_tokens_map.json',
 './fake-news-multilingual\\sentencepiece.bpe.model',
 './fake-news-multilingual\\added_tokens.json',
 './fake-news-multilingual\\tokenizer.json')

In [20]:
# Evaluate Model
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.2555449903011322, 'eval_runtime': 230.2998, 'eval_samples_per_second': 1.719, 'eval_steps_per_second': 0.217, 'epoch': 3.0}


In [26]:
from sklearn.metrics import accuracy_score
import numpy as np

# Evaluate Model
eval_results = trainer.evaluate()

# Get predictions
predictions = trainer.predict(val_data)

# Extract logits and labels
logits, labels, _ = predictions

# Get predicted classes
preds = np.argmax(logits, axis=1)

# Calculate accuracy
accuracy = accuracy_score(labels, preds)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9217


In [7]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the model and tokenizer
model_name = "./fake-news-multilingual
"  # Change this to your fine-tuned model path if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to predict the label
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    label = "Fake News" if predicted_class == 1 else "Real News"
    return label

# Get user input and predict
while True:
    text = input("Enter text to classify (or type 'exit' to quit): ")
    if text.lower() == "exit":
        break
    prediction = predict(text)
    print(f"Prediction: {prediction}")

Enter text to classify (or type 'exit' to quit):  	text 641	பாராளுமன்றத்தின் வெல்ஷ் உறுப்பினர்களின் கூட்டத்தின் போது, ​​பிரதமர் நைகல் எவன்ஸ் ட்ரம்ப் எதிர்ப்பு உறுப்பினர்களை வெட்கினார், அவர்கள் தங்கள் சொந்த ஜனநாயக பதிப்பை முடிவு செய்துள்ளனர் என்பதையும், 61 மில்லியன் அமெரிக்கர்கள் டொனால்ட் டிரம்பை வாக்குப் பெட்டியில் ஆதரித்தார்கள் என்பதையும் நினைவூட்டுகிறது.அவர் அவர்களிடம் சொல்வதன் மூலம் தொடங்கினார்: டொனால்ட் டிரம்பிற்கு அமெரிக்க மக்கள் வாக்களித்தார்கள் என்பதைப் புரிந்துகொள்வது கடினம் என்று கருதுபவர்களுக்கு, அவர் அமெரிக்காவின் ஜனாதிபதியாக இருப்பதால் அதைப் பெறுங்கள்.பிரெக்ஸிட் பிரிட்டிஷ் வாக்காளர்களால் மறந்துவிட்ட பாராளுமன்ற உறுப்பினர்களிடம் வழங்கப்பட்ட வாக்கெடுப்பு என்று பாராளுமன்ற உறுப்பினர்களுக்கு எவன்ஸ் நினைவுபடுத்தினார்.நாம் நம்மை நாமே கேட்டுக்கொள்ள வேண்டும், இது உண்மையில் என்னை உள்ளடக்கியது, நாம் நம்மை நாமே கேட்டுக்கொள்ள வேண்டும், மக்கள் ஏன் தங்களிடம் உள்ள ஜனநாயக முடிவுகளை எடுத்தார்கள் என்று மக்கள் ஏன் விட்டுவிட்டார்கள்?இவர்கள் மறந்துபோனவர்கள், ஐக்கிய இராச்சியத்தில் மறந்துபோனவர்களைப

Prediction: Fake News


Enter text to classify (or type 'exit' to quit):  	text 638	அங்காரா (ராய்ட்டர்ஸ்) - ரஷ்யாவின் சோச்சிக்கு உத்தியோகபூர்வ விஜயத்தின் போது ரஷ்ய ஜனாதிபதி விளாடிமிர் புடினுடன் திட்டமிட்ட மாஸ்கோ ஆதரவு சிரிய காங்கிரஸை விவாதிப்பதாக துருக்கிய ஜனாதிபதி தயிப் எர்டோகன் திங்களன்று தெரிவித்தார்.ரஷ்யா, குவைத் மற்றும் கத்தார் ஆகியோருக்கு புறப்படுவதற்கு முன்பு செய்தியாளர்களிடம் பேசிய எர்டோகன், ரஷ்யாவுடன் விசா இல்லாத பயணத்திற்கு தடைகளை நீக்குமாறு புடினிடம் கேட்பேன் என்றும் கூறினார்.தேசிய உரையாடலில் ரஷ்ய நிதியுதவி அளித்த சிரிய காங்கிரஸ், நவம்பர் மாதத்தில் திட்டமிடப்பட்டுள்ளது.18, ஒத்திவைக்கப்பட்டுள்ளது, துருக்கியின் ஆட்சேபனைகளுக்குப் பிறகு பிரதான சிரிய குர்திஷ் குழு அழைக்கப்படாது என்று எர்டோகன் செய்தித் தொடர்பாளர் கடந்த வாரம் தெரிவித்தார்.


Prediction: Real News


Enter text to classify (or type 'exit' to quit):  	text 664	మా ఇమామ్ యొక్క చీఫ్ 2007 లో అతన్ని ఎన్నుకోవాలనుకున్నప్పుడు అతను క్రైస్తవుడని యునైటెడ్ స్టేట్స్కు చెప్పాడు.అతని నల్లజాతి ఉదారవాద బోధకుడు బరాక్ ఒక సామాజిక నిర్వాహకుడిగా తన స్థితిని పెంచే మార్గంగా మాత్రమే చర్చిని ఉపయోగించాడు. అమెరికన్లకు వ్యతిరేకంగా చాలా దారుణం. ఒక దశాబ్దానికి పైగా, మరియు కొన్ని పరిస్థితులలో కనుగొనటానికి సహాయపడింది.అధ్యక్షుడు బాల్డిమోర్ శనివారం మరియు వైట్ హౌస్ శనివారం ధృవీకరించారు. ముస్లిం అమెరికన్లు మన దేశాన్ని యూనిఫాం మరియు మా పురుషులు మరియు మహిళా యూనిఫాంలో రక్షిస్తారు. ముస్లిం బ్రదర్‌హుడ్ సభ్యుడు, సంస్థను పద్దెనిమిది సంవత్సరాలుగా మార్చారు.యునైటెడ్ స్టేట్స్లో సౌదీ అరేబియా రాయబార కార్యాలయం నుండి పెద్ద సబ్సిడీకి ధార్ అల్-హిజ్రా మసీదు స్థాపించబడింది, ఇది 5,000 మంది ముస్లింలకు వసతి కల్పించడానికి వీలు కల్పించింది.వాషింగ్టన్, డిసి వెలుపల ఉన్న ధార్ అల్-హిజ్రా, ప్రార్థన చేసిన అనేక మంది ఉన్నత స్థాయి ఇస్లామిక్ ఉగ్రవాదులతో ముడిపడి ఉంది, దీనిలో ప్రధాన హసన్ ఫిడేల్ హుడ్ హత్య జిహాదీ, సెప్టెంబర్ 11 కిడ్నాపర్లు మరియు 1993 ప్రపం

...

Enter text to classify (or type 'exit' to quit):  	text 10420	यदि अमेरिकी राज्य विभाग 16 मार्च से शुरू होने के लिए 120-दिन के ठहराव की तैयारी कर रहा था, तो यह कैसे है कि यह बड़ी संख्या में शरणार्थी विमानों के लिए तैयार थे?क्या कोई प्रभारी (कैरियर के लोगों के अलावा) जनसंख्या, शरणार्थियों और प्रवास ब्यूरो में है?क्या वे अभी भी सभी शॉट्स कह रहे हैं?या, क्या यह संभव है कि व्हाइट हाउस ने कार्यकारी आदेश के इस हिस्से पर लड़ने के लिए तैयार नहीं किया (यह मानते हुए कि हम टी नोटिस नहीं करेंगे)? 342 नए आगमन के बीच शीर्ष पांच राष्ट्रीयताएं इस प्रकार हैं: सीरिया (55 और 51 उनमें से थे।मुस्लिम) सोमालिया (50 और सभी मुस्लिम हैं) बर्मा (44 और उन 17 की आश्चर्यजनक रूप से उच्च संख्या मुस्लिम हैं) इराक (41 और 32 मुस्लिम हैं)


Prediction: Fake News


Enter text to classify (or type 'exit' to quit):  quit


Prediction: Fake News


Enter text to classify (or type 'exit' to quit):  exit
